## 1.了解yaml

##### 它使用空白符号缩进和大量依赖外观的特色，特别适合用来表达或编辑数据结构、各种配置文件、倾印调试内容、文件大纲
我们用yaml做配置文件


In [4]:
import yaml #导入yaml包之后，就可以加载yaml文件并解析它
import os

In [5]:
#读取yaml文件，获取数据，并转换成字典
data = yaml.load(open('./hlcm.yaml','r',encoding="utf-8"))
data

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


{'name': 'Household Location Choice Model',
 'model_type': 'segmented_discretechoice',
 'segmentation_col': 'income_quartile',
 'choosers_fit_filters': None,
 'choosers_predict_filters': None,
 'alts_fit_filters': ["general_type == 'Residential'"],
 'alts_predict_filters': ["general_type == 'Residential'"],
 'interaction_predict_filters': None,
 'choice_column': None,
 'sample_size': 100,
 'estimation_sample_size': 3000,
 'prediction_sample_size': None,
 'default_config': {'model_expression': 'np.log1p(residential_sales_price) + np.log1p(unit_sqft) + sum_residential_units + ave_unit_sqft + ave_lot_sqft + ave_income + hhsize + jobs + sfdu + renters + poor + population - 1'},
 'models': {0: {'fit_parameters': {'Coefficient': {'ave_income': -0.1803739241436568,
     'ave_lot_sqft': -0.14524735635397554,
     'ave_unit_sqft': 0.5248733590599849,
     'hhsize': -0.7902166295229691,
     'jobs': 0.048439041766531486,
     'np.log1p(residential_sales_price)': -0.03454228784548852,
     'np.lo

In [6]:
#获取data里面的数据
data['alts_fit_filters']

["general_type == 'Residential'"]

## 2. 需要的一些函数

##### yaml_to_class

In [35]:
from urbansim.models import RegressionModel, SegmentedRegressionModel, \
    MNLDiscreteChoiceModel, SegmentedMNLDiscreteChoiceModel, \
    GrowthRateTransition
def yaml_to_class(cfg):
    import yaml
    model_type = yaml.load(open(cfg))["model_type"]
    return {
        "regression": RegressionModel,
        "segmented_regression": SegmentedRegressionModel,
        "discretechoice": MNLDiscreteChoiceModel,
        "segmented_discretechoice": SegmentedMNLDiscreteChoiceModel
    }[model_type]
yaml_to_class('./hlcm.yaml')

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


urbansim.models.dcm.SegmentedMNLDiscreteChoiceModel

##### cfg

In [37]:
cfg = './hlcm.yaml'
cfg = yaml_to_class(cfg).from_yaml(str_or_buffer = cfg)
cfg
#这个from_yaml 是urbansim.models.dcm.SegmentedMNLDiscreteChoiceModel的from_yaml()函数

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


##### column_list

In [16]:
import numpy as np
import pandas as pd
import toolz as tz
def column_list(tables, columns):
    """
返回tables中位于cfg使用列的列名，就是又存在于tables中，又存在与cfg中的
    """
    columns = set(columns)
    foundcols = tz.reduce(lambda x, y: x.union(y), (set(t.columns) for t in tables))
    return list(columns.intersection(foundcols))

df_households = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\households_包含需要进行区位选择的家庭.csv')

cfg_columns_used =  ['general_type',
     'residential_sales_price',
     'unit_sqft',
     'sum_residential_units',
     'ave_unit_sqft',
     'ave_lot_sqft',
     'ave_income',
     'hhsize',
     'jobs',
     'sfdu',
     'renters',
     'poor',
     'population',
     'income_quartile']
print(df_households.columns)
cfg_columns = cfg_columns_used
column_list([df_households], cfg_columns) #处于df_households，又处于cfg中的

Index(['household_id', 'building_id', 'building_type_id', 'income', 'persons',
       'tenure', 'income_quartile', 'zone_id'],
      dtype='object')


['income_quartile']

##### from_yaml()从保存的yaml配置创建分段MNLCM 模型实例

In [17]:
tables = [df_households]
cfg_columns_used =  ['general_type',
     'residential_sales_price',
     'unit_sqft',
     'sum_residential_units',
     'ave_unit_sqft',
     'ave_lot_sqft',
     'ave_income',
     'hhsize',
     'jobs',
     'sfdu',
     'renters',
     'poor',
     'population',
     'income_quartile']
additional_columns = ["building_id"]
columns = column_list(tables, cfg_columns)+additional_columns
columns  #这个就是我们需要的列名（to_frame ）

['income_quartile', 'building_id']

## 3. orca使用
用orca运行我们的数据<br>
有计算列的基础表，已经准备好的<br>
目前的目标是生成模型所需要的数据表<br>
下面主要是想获取区位选择模型中的关键表格locations_df

In [21]:
#先清除一下缓冲区
import orca
orca.orca.clear_all()

##### 读取基础表（DataFrame格式）

In [22]:
buildings = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\区位选择的基础建筑物表.csv',index_col=0)
nodes = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\区位选择的基础表zones.csv',index_col=0)
households = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\households_包含需要进行区位选择的家庭.csv',index_col=0)
print(nodes.columns)
nodes.columns=['gid', 'tract', 'area', 'acres', 'sum_residential_units',
       'sum_job_spaces', 'population', 'jobs', 'ave_lot_sqft', 'ave_income',
       'hhsize', 'ave_unit_sqft', 'sfdu', 'poor', 'renters', 'zone_id']
# print(nodes.columns)

Index(['gid', 'tract', 'area', 'acres', 'sum_residential_units',
       'sum_job_spaces', 'population', 'jobs', 'ave_lot_sqft', 'ave_income',
       'hhsize', 'ave_unit_sqft', 'sfdu', 'poor', 'renters', 'zone_id.1'],
      dtype='object')


##### add_table

In [26]:
#把我们的基础表buildings、zones、households(DataFrame格式)添加到orca中
orca.add_table('buildings',buildings)
orca.add_table('zones',nodes)
orca.add_table('households',households)

##### 设置表格之间的关系

In [28]:
orca.broadcast('zones', 'buildings', cast_index=True, onto_on='zone_id')

##### 读取表格
get_table()

In [32]:
choosers = orca.get_table('households')
nodes = orca.get_table('zones')
buildings = orca.get_table('buildings')
type(choosers)  #可以看出获取的表的格式是orca.orca.DataFrameWrapper

orca.orca.DataFrameWrapper

##### to_frame()
将orca.orca.DataFrameWrapper生成为一个DataFrame

In [38]:
choosers.to_frame() #可以看看出

,building_id,building_type_id,income,persons,tenure,income_quartile,zone_id
household_id,,,,,,,
1,136325,1,27310,1,2,0,38.0
4,133982,2,25000,1,2,0,44.0
5,25633,1,3100,1,1,0,170.0
6,132596,2,7200,1,1,0,45.0
7,57108,1,27310,1,2,0,164.0
...,...,...,...,...,...,...,...
2630612,74346,3,690,1,1,0,148.0
2630613,78154,3,24650,1,1,0,141.0
2630614,-1,3,58000,1,1,1,NaN


##### cfg_columns_used
前面已经计算的，指的是配置文件中写的需要用到的列（主要是model_expression中的变量名）

In [41]:
cfg_columns_used

['general_type',
 'residential_sales_price',
 'unit_sqft',
 'sum_residential_units',
 'ave_unit_sqft',
 'ave_lot_sqft',
 'ave_income',
 'hhsize',
 'jobs',
 'sfdu',
 'renters',
 'poor',
 'population',
 'income_quartile']

In [42]:
#生成location_df 额外需要的列
supply_fname = "residential_units" #有计算列建筑物中的列
vacant_fname =  "vacant_residential_units" #有计算列建筑物中的列
additional_columns = [supply_fname,vacant_fname]

##### columns 
即在cfg列又在buildings列中的列+额外列


In [46]:
tables = [buildings, nodes]
columns = column_list(tables, cfg_columns)+additional_columns
columns #一共有15列（cfg和buildings列重复的有13个+额外的2列）

['ave_income',
 'ave_unit_sqft',
 'jobs',
 'unit_sqft',
 'renters',
 'population',
 'residential_sales_price',
 'general_type',
 'sfdu',
 'poor',
 'sum_residential_units',
 'ave_lot_sqft',
 'hhsize',
 'residential_units',
 'vacant_residential_units']

##### tables

In [52]:
tables  = [t for t in tables if t is not None]  #将两个df组合成一个list,就是list中嵌套两个df

In [53]:
type(tables)

list

In [54]:
len(tables) 

2

In [59]:

locations_df = orca.merge_tables(target=tables[0].name,
                               tables=tables, columns=columns)
locations_df

,zone_id,zone_id_buildings,unit_sqft,residential_units,vacant_residential_units,general_type,residential_sales_price,zone_id_zones,ave_unit_sqft,ave_income,jobs,renters,population,sfdu,poor,sum_residential_units,ave_lot_sqft,hhsize
building_id,,,,,,,,,,,,,,,,,,
66515,103.0,103.0,1585.0000,3,0.0,Residential,360,103,7.153834,11.156265,6.259581,7.532624,8.556991,1.386294,6.859615,7.951559,7.293847,1.098612
65547,103.0,103.0,3314.0000,1,0.0,Residential,96,103,7.153834,11.156265,6.259581,7.532624,8.556991,1.386294,6.859615,7.951559,7.293847,1.098612
66514,103.0,103.0,1650.0000,3,-1.0,Residential,360,103,7.153834,11.156265,6.259581,7.532624,8.556991,1.386294,6.859615,7.951559,7.293847,1.098612
66655,103.0,103.0,941.5000,8,1.0,Residential,845,103,7.153834,11.156265,6.259581,7.532624,8.556991,1.386294,6.859615,7.951559,7.293847,1.098612
65727,103.0,103.0,739.6667,6,0.0,Residential,1285,103,7.153834,11.156265,6.259581,7.532624,8.556991,1.386294,6.859615,7.951559,7.293847,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855320,14.0,14.0,170061.0000,1,1.0,Office,0,14,7.696667,10.477316,9.366831,1.098612,4.007333,0.000000,3.295837,6.131226,9.034048,0.693147
1854424,14.0,14.0,996760.0000,1,1.0,Office,0,14,7.696667,10.477316,9.366831,1.098612,4.007333,0.000000,3.295837,6.131226,9.034048,0.693147
1853798,14.0,14.0,523385.0000,1,1.0,Office,0,14,7.696667,10.477316,9.366831,1.098612,4.007333,0.000000,3.295837,6.131226,9.034048,0.693147


In [60]:
# def to_frame(tables, cfg, additional_columns=[]):
#     cfg = yaml_to_class(cfg).from_yaml(str_or_buffer=cfg)
#     tables = [t for t in tables if t is not None]
#     columns = misc.column_list(tables, cfg.columns_used()) + additional_columns
#     if len(tables) > 1:
#         df = orca.merge_tables(target=tables[0].name,
#                                tables=tables, columns=columns)
#     else:
#         df = tables[0].to_frame(columns)
#     df = deal_with_nas(df)
#     return df

"""
上面的操作主要都可以基于to_frame函数

"""
# locations_df = to_frame([buildings, nodes], cfg,
#                             [supply_fname, vacant_fname])
# locations_df

'\n上面的操作主要都可以基于to_frame函数\n\n'